In [1]:
# To import functions from utils.py, which is in the parent diretory

import sys
import os

current_dir = os.path.dirname(os.path.abspath('__file__')) 

parent_dir = os.path.dirname(current_dir)

sys.path.append(parent_dir)

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from utils import delete_patients_with_the_same_GUID, delete_patients_without_GUID, averaging_scores

sns.set_style("whitegrid")

We observed previously that there are patients with different `participant_id` yet with the same `GUID`, and that these patients are likely to be the same person, since their age only increased 1 after 1 year and their updrs scores are rather close. So we decide to remove observations if they correspond to only 1 `GUID` but more than two different `participant_id`'s. The following codes execute this idea for targets and all potential features.

This is a demonstration of `delete_patients_with_the_same_GUID` and `delete_patients_without_GUID` function for the `updrs_1` table.

In [5]:
updrs_1=pd.read_csv("../new-data/releases_2023_v4release_1027_clinical_MDS_UPDRS_Part_I.csv")

Observe

In [6]:
updrs_1_cleaned=delete_patients_with_the_same_GUID(updrs_1)

Sanity check for delete_patients_with_the_same_GUID

In [8]:
groups=updrs_1_cleaned.groupby(by="GUID")
for i,df in groups:
    if len(df.participant_id.unique())>=2:
        print(df)

There is nothing printed out, meaning there is not a GUID which corresponds to 2 or more participant ids.

Also, observe that there are patients who took a updrs assessment test multiple times in a single month or during the screening time. Since the base unit for our time series analysis is month, we are going to average them so that each patient has at most one score for each month.

In [ ]:
updrs_1=pd.read_csv("../new-data/releases_2023_v4release_1027_clinical_MDS_UPDRS_Part_I.csv")
updrs_1_cleaned=delete_patients_with_the_same_GUID(updrs_1)
updrs_1_cleaned=averaging_scores(updrs_1_cleaned, ["mds_updrs_part_i_summary_score"])

Processing screening records by participant...
Consolidated 1009 screening records into 1005 baseline records.
Processing regular visit records by participant and month...
Consolidated 19444 regular records into 19434 records.
Processing screening records by participant...
Consolidated 319 screening records into 319 baseline records.
Processing regular visit records by participant and month...
Consolidated 12314 regular records into 12309 records.


In [ ]:
updrs_2=pd.read_csv("../new-data/releases_2023_v4release_1027_clinical_MDS_UPDRS_Part_II.csv")
updrs_2_cleaned=delete_patients_with_the_same_GUID(updrs_2)
updrs_2_cleaned=averaging_scores(updrs_2_cleaned, ["mds_updrs_part_ii_summary_score"])

Processing screening records by participant...
Consolidated 1004 screening records into 1000 baseline records.
Processing regular visit records by participant and month...
Consolidated 20366 regular records into 20356 records.
Processing screening records by participant...
Consolidated 319 screening records into 319 baseline records.
Processing regular visit records by participant and month...
Consolidated 13240 regular records into 13235 records.


In [ ]:
updrs_3=pd.read_csv("../new-data/releases_2023_v4release_1027_clinical_MDS_UPDRS_Part_III.csv")
updrs_3_cleaned=delete_patients_with_the_same_GUID(updrs_3)
updrs_3_cleaned=averaging_scores(updrs_3_cleaned, ["mds_updrs_part_iii_summary_score"])

Processing screening records by participant...
Consolidated 1044 screening records into 1020 baseline records.
Processing regular visit records by participant and month...
Consolidated 22500 regular records into 20478 records.
Processing screening records by participant...
Consolidated 319 screening records into 318 baseline records.
Processing regular visit records by participant and month...
Consolidated 14343 regular records into 13415 records.


In [ ]:
updrs_4=pd.read_csv("../new-data/releases_2023_v4release_1027_clinical_MDS_UPDRS_Part_IV.csv")
updrs_4_cleaned=delete_patients_with_the_same_GUID(updrs_4)


caffeine_data=pd.read_csv("../new-data/releases_2023_v4release_1027_clinical_Caffeine_history.csv")
caffeine_data_cleaned=delete_patients_with_the_same_GUID(caffeine_data)


demographic_data=pd.read_csv("../new-data/releases_2023_v4release_1027_clinical_Demographics.csv")
demographic_data_cleaned=delete_patients_with_the_same_GUID(demographic_data)


family_data=pd.read_csv("../new-data/releases_2023_v4release_1027_clinical_Family_History_PD.csv")
family_data_cleaned=delete_patients_with_the_same_GUID(family_data)


DaTSCAN_SBR_data=pd.read_csv("../new-data/releases_2023_v4release_1027_clinical_DaTSCAN_SBR.csv")
DaTSCAN_SBR_data_cleaned=delete_patients_with_the_same_GUID(DaTSCAN_SBR_data)


DTI_data=pd.read_csv("../new-data/releases_2023_v4release_1027_clinical_DTI.csv")
DTI_data_cleaned=delete_patients_with_the_same_GUID(DTI_data)

In [ ]:
updrs_1_cleaned=updrs_1_cleaned[["participant_id", "visit_name", "visit_month", "mds_updrs_part_i_summary_score"]]

updrs_2_cleaned=updrs_2_cleaned[["participant_id", "visit_name", "visit_month", "mds_updrs_part_ii_summary_score"]]

updrs_3_cleaned=updrs_3_cleaned[["participant_id", "visit_name", "visit_month", "mds_updrs_part_iii_summary_score"]]

We first join the target variables, `updrs_1`, `updrs_2` and `updrs_3` in an inner way, so that all participant will have a complete record of all the three scores.  

(Need to have a more convincing reason, maybe state the reason from the correlation of updrs scores)

In [ ]:
cleaned_updrs_scores=updrs_1_cleaned.merge(
    updrs_2_cleaned, 
    on=["participant_id", "visit_name", "visit_month"], 
    how="inner").merge(
        updrs_3_cleaned,
        on=["participant_id", "visit_name", "visit_month"], 
        how="inner")

C:\Users\smile\AppData\Local\Temp\ipykernel_16700\2721668787.py:4: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  how="inner").merge(
C:\Users\smile\AppData\Local\Temp\ipykernel_16700\2721668787.py:11: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  how="inner").merge(


In [ ]:
cleaned_updrs_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20319 entries, 0 to 20318
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   participant_id                    20319 non-null  object 
 1   visit_name                        20319 non-null  object 
 2   visit_month                       20319 non-null  int64  
 3   mds_updrs_part_i_summary_score    20319 non-null  float64
 4   mds_updrs_part_ii_summary_score   20317 non-null  float64
 5   mds_updrs_part_iii_summary_score  20316 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 952.6+ KB


Before we merge the features with the targets, we need to check whether a feature is a time dependent feature, as time-dependent variables and non-time-dependent variables are treated differently in model with lags.

So we take a look at whether a test or a survey is conducted several times, and also use our common knowledge.

In [ ]:
display(caffeine_data_cleaned.info())


print("----------------------------------")
print("Number of times every participant attended the survey")
display(caffeine_data_cleaned.participant_id.value_counts().value_counts())

print("----------------------------------")
print("Detailed time of the survey:")
display(caffeine_data_cleaned.visit_month.value_counts())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155 entries, 0 to 2154
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   participant_id                   2155 non-null   object 
 1   GUID                             1453 non-null   object 
 2   visit_name                       2155 non-null   object 
 3   visit_month                      1418 non-null   float64
 4   caff_drinks_ever_used_regularly  2155 non-null   object 
 5   caff_drinks_current_use          2063 non-null   object 
dtypes: float64(1), object(5)
memory usage: 101.1+ KB


None

----------------------------------
Number of times every participant attended the survey


count
1    2155
Name: count, dtype: int64

----------------------------------
Detailed time of the survey:


visit_month
 0.0     1390
 12.0      11
-1.0       10
 24.0       3
 36.0       2
 48.0       1
 60.0       1
Name: count, dtype: int64

The last result means that every participant is only asked once about their caffeine usage (current and previous), so this feature is not time dependent, which also matches with our intuition. So we drop the `visit_name` and `visit_month` and match the caffeine record with the feature by `participant_id`. We use `how="left"` so that we won't include additional patient who does not have a updrs score record. 

In [ ]:
caffeine_data_cleaned=caffeine_data_cleaned.drop(["GUID","visit_name","visit_month"],axis=1)

In [ ]:
caffeine_data_cleaned.head(5)

,participant_id,caff_drinks_ever_used_regularly,caff_drinks_current_use
0,HB-PD_INVAA223GY7,No,No
1,HB-PD_INVAB465GYE,Yes,Yes
2,HB-PD_INVAD033HX2,No,No
3,HB-PD_INVAD802MY3,No,No
4,HB-PD_INVAD946MJ7,Yes,Yes


In [ ]:
cleaned_data=cleaned_updrs_scores.merge(
    caffeine_data_cleaned,
    on="participant_id", 
    how="left"
)
cleaned_data

,participant_id,visit_name,visit_month,mds_updrs_part_i_summary_score,mds_updrs_part_ii_summary_score,mds_updrs_part_iii_summary_score,caff_drinks_ever_used_regularly,caff_drinks_current_use
0,LC-460010,SC,-1,1.0,0.0,3.0,NaN,NaN
1,PP-10874,SC,-1,0.0,0.0,2.0,NaN,NaN
2,PP-12499,SC,-1,4.0,2.0,0.0,NaN,NaN
3,PP-12593,SC,-1,4.0,3.0,2.0,Yes,Yes
4,PP-13039,SC,-1,5.0,3.0,0.0,Yes,Yes
...,...,...,...,...,...,...,...,...
20314,SY-PDZX943HWN,M36,36,10.0,5.0,10.0,NaN,NaN
20315,SY-PDZY968RFA,M0,0,2.0,0.0,11.0,NaN,NaN
20316,SY-PDZY968RFA,M36,36,3.0,5.0,22.0,NaN,NaN
20317,SY-PDZZ260HUM,M0,0,3.0,1.0,13.0,NaN,NaN


We repeat the above process with other potential features.

In [ ]:
display(demographic_data_cleaned.info())


print("----------------------------------")
print("Number of times every participant attended the survey")
display(demographic_data_cleaned.participant_id.value_counts().value_counts())

print("----------------------------------")
print("Detailed time of the survey:")
display(demographic_data_cleaned.visit_month.value_counts())

<class 'pandas.core.frame.DataFrame'>
Index: 10898 entries, 0 to 10907
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   participant_id         10898 non-null  object
 1   GUID                   4403 non-null   object
 2   visit_name             10898 non-null  object
 3   visit_month            10898 non-null  int64 
 4   age_at_baseline        10898 non-null  int64 
 5   sex                    10898 non-null  object
 6   ethnicity              6293 non-null   object
 7   race                   10879 non-null  object
 8   education_level_years  6312 non-null   object
dtypes: int64(2), object(7)
memory usage: 851.4+ KB


None

----------------------------------
Number of times every participant attended the survey


count
1    10898
Name: count, dtype: int64

----------------------------------
Detailed time of the survey:


visit_month
 0     8912
-1     1979
 48       3
 24       2
 12       1
 36       1
Name: count, dtype: int64

The last result also confirms with our intuition that this feature is not time dependent. So we drop the `visit_name` and `visit_month` and match the demographic record with the feature by `participant_id`. We use `how="left"` so that we won't include additional patient who does not have a updrs score record. 

Also observe that `race` carries more information than `ethnicity`, so we drop `ethnicity`.

In [ ]:
demographic_data_cleaned=demographic_data_cleaned.drop(["GUID","visit_name","visit_month", "ethnicity"],axis=1)

In [ ]:
cleaned_data=cleaned_data.merge(
    demographic_data_cleaned,
    on="participant_id", 
    how="left"
)

In [ ]:
display(family_data_cleaned.info())


print("----------------------------------")
print("Number of times every participant attended the survey")
display(family_data_cleaned.participant_id.value_counts().value_counts())

print("----------------------------------")
print("Detailed time of the survey:")
display(family_data_cleaned.visit_month.value_counts())

<class 'pandas.core.frame.DataFrame'>
Index: 8131 entries, 0 to 8138
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   participant_id             8131 non-null   object 
 1   GUID                       3790 non-null   object 
 2   visit_name                 8131 non-null   object 
 3   visit_month                7006 non-null   float64
 4   biological_mother_with_pd  8079 non-null   object 
 5   biological_father_with_pd  8077 non-null   object 
 6   other_relative_with_pd     8097 non-null   object 
dtypes: float64(1), object(6)
memory usage: 508.2+ KB


None

----------------------------------
Number of times every participant attended the survey


count
1    8087
2      22
Name: count, dtype: int64

----------------------------------
Detailed time of the survey:


visit_month
 0.0     6153
-1.0      834
 84.0       9
 72.0       6
 96.0       4
Name: count, dtype: int64

We found that there are 22 people who are asked about their family history twice. Although that is a small number, we take a look at that.

In [ ]:
groups = family_data_cleaned.groupby(by="participant_id")

print("Checking for participants with inconsistent survey answers...\n")

for participant_id, df in groups:
    
    if len(df) >= 2:
        
        
        mother_answers_changed = df['biological_mother_with_pd'].nunique() > 1
        father_answers_changed = df['biological_father_with_pd'].nunique() > 1
        other_relatives_answers_changed = df['other_relative_with_pd'].nunique() > 1

        print("This patient was asked about their family history at:")
        print(df.visit_name.values[::-1])
        
  
        if mother_answers_changed or father_answers_changed or other_relatives_answers_changed:
            
            print(f"--- Participant {participant_id} answered differently ---")
            
            print("\nHis answers are:")
            
            print("On the mother side:")
            print(df.biological_mother_with_pd.values[::-1])
            
            print("On the father side:")
            print(df.biological_father_with_pd.values[::-1])
            
            print("Other relatives:")
            print(df.other_relative_with_pd.values[::-1])
            print("----------------------------------")

        

Checking for participants with inconsistent survey answers...

This patient was asked about their family history at:
['SC' 'M96']
This patient was asked about their family history at:
['SC' 'M96']
This patient was asked about their family history at:
['SC' 'M96']
This patient was asked about their family history at:
['SC' 'M96']
This patient was asked about their family history at:
['SC' 'M84']
--- Participant PP-3018 answered differently ---

His answers are:
On the mother side:
['No' 'No']
On the father side:
['No' 'Yes']
Other relatives:
['Yes' 'Yes']
----------------------------------
This patient was asked about their family history at:
['SC' 'M84']
This patient was asked about their family history at:
['SC' 'M84']
This patient was asked about their family history at:
['SC' 'M84']
This patient was asked about their family history at:
['SC' 'M84']
This patient was asked about their family history at:
['SC' 'M84']
This patient was asked about their family history at:
['SC' 'M84']
Th

We found that only two patients had different answers over time. In both of these cases, the change reflected an update from "No" to "Yes". Given this, and the strong genetic component of Parkinson's disease, we applied a rule to assign "Yes" to a variable if a participant ever reported it as such. This approach ensures we use the most complete affirmative information provided.

In [ ]:
def consolidate_history(group):
    """
    Consolidates multiple family history records for a single participant
    into one record based on the "if any is Yes, then Yes" rule.
    
    It also keeps the visit information from the most recent record.
    """

    latest_record = group.sort_values('visit_month', ascending=False).iloc[0]
    

    mother_history = 'Yes' if 'Yes' in group['biological_mother_with_pd'].values else 'No'
    father_history = 'Yes' if 'Yes' in group['biological_father_with_pd'].values else 'No'
    other_history = 'Yes' if 'Yes' in group['other_relative_with_pd'].values else 'No'
    
    consolidated_data = {
        'participant_id': group['participant_id'].iloc[0],
        'GUID': group['GUID'].iloc[0],
        'visit_name': latest_record['visit_name'],
        'visit_month': latest_record['visit_month'],
        'biological_mother_with_pd': mother_history,
        'biological_father_with_pd': father_history,
        'other_relative_with_pd': other_history
    }
    
    return pd.Series(consolidated_data)

try:
    family_data_cleaned = family_data_cleaned.groupby('participant_id').apply(consolidate_history).reset_index(drop=True)

except FileNotFoundError:
    print("Error: The file 'releases_2023_v4release_1027_clinical_Family_History_PD.csv' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

C:\Users\smile\AppData\Local\Temp\ipykernel_16700\3283215029.py:29: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  family_data_cleaned = family_data_cleaned.groupby('participant_id').apply(consolidate_history).reset_index(drop=True)


In [ ]:
family_data_cleaned=family_data_cleaned.drop(["GUID","visit_name","visit_month"],axis=1)

In [ ]:
cleaned_data=cleaned_data.merge(
    family_data_cleaned,
    on="participant_id", 
    how="left"
)

In [ ]:
display(DaTSCAN_SBR_data_cleaned.info())


print("----------------------------------")
print("Number of times every participant attended the survey")
display(DaTSCAN_SBR_data_cleaned.participant_id.value_counts().value_counts())

print("----------------------------------")
print("Detailed time of the survey:")
display(DaTSCAN_SBR_data_cleaned.visit_month.value_counts())

print("----------------------------------")
print("Any repeated test in a single month or during screening? (#2, #3, etc. indicates repeated visits.)")
display(DaTSCAN_SBR_data_cleaned.visit_name.value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   participant_id  2280 non-null   object 
 1   GUID            881 non-null    object 
 2   visit_name      2280 non-null   object 
 3   visit_month     2276 non-null   float64
 4   sbr_caudate_r   2280 non-null   float64
 5   sbr_caudate_l   2280 non-null   float64
 6   sbr_putamen_r   2280 non-null   float64
 7   sbr_putamen_l   2280 non-null   float64
dtypes: float64(5), object(3)
memory usage: 142.6+ KB


None

----------------------------------
Number of times every participant attended the survey


count
1    431
4    271
2    167
3    128
5      7
6      2
Name: count, dtype: int64

----------------------------------
Detailed time of the survey:


visit_month
-1.0     925
 24.0    533
 12.0    381
 48.0    345
 0.0      73
 36.0     11
 60.0      7
 30.0      1
Name: count, dtype: int64

----------------------------------
Any repeated test in a single month or during screening? (#2, #3, etc. indicates repeated visits.)


visit_name
SC     925
M24    533
M12    381
M48    345
M0      73
M36     11
M60      7
LOG      4
M30      1
Name: count, dtype: int64

Since there is a significant number of patients who took the test several times, and that these characters are also changing with time, we treat it as a time-dependent variable. So we keep the `visit_month` column and merge that with the target and feature table with `on=["participant_id","visit_month"]`. Since all `visit_name` don't carry `#02` or `#03`, there is also no repeated test during screening or a single month. (Need literal revision)

Note that there are 4 `visit_month` values missing.

In [ ]:
DaTSCAN_SBR_data_cleaned[DaTSCAN_SBR_data_cleaned.visit_month.isna()]

,participant_id,GUID,visit_name,visit_month,sbr_caudate_r,sbr_caudate_l,sbr_putamen_r,sbr_putamen_l
597,PP-3356,NaN,LOG,NaN,2.51,2.54,2.19,2.26
1158,PP-3638,NIHNA340EADWL,LOG,NaN,1.68,1.88,0.56,0.58
1975,PP-41281,NaN,LOG,NaN,0.69,0.57,0.12,0.24
1978,PP-41287,NaN,LOG,NaN,1.81,2.06,0.75,0.63


Observe that they all come from records without visit, so we drop them.

In [ ]:
DaTSCAN_SBR_data_cleaned=DaTSCAN_SBR_data_cleaned.dropna(subset="visit_month")

In [ ]:
DaTSCAN_SBR_data_cleaned=DaTSCAN_SBR_data_cleaned.drop(["GUID","visit_name"],axis=1)

In [ ]:
cleaned_data=cleaned_data.merge(
    DaTSCAN_SBR_data_cleaned,
    on=["participant_id","visit_month"],
    how="left"
)

In [ ]:
display(DTI_data_cleaned.info())


print("----------------------------------")
print("Number of times every participant attended the survey")
display(DTI_data_cleaned.participant_id.value_counts().value_counts())

print("----------------------------------")
print("Detailed time of the survey:")
display(DTI_data_cleaned.visit_month.value_counts())

print("----------------------------------")
print("Any repeated test in a single month or during screening? (#2, #3, etc. indicates repeated visits.)")
display(DTI_data_cleaned.visit_name.value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1196 entries, 0 to 1195
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   participant_id        1196 non-null   object 
 1   GUID                  364 non-null    object 
 2   visit_name            1196 non-null   object 
 3   visit_month           0 non-null      float64
 4   dti_brain_tissue      1196 non-null   object 
 5   dti_measure           1196 non-null   object 
 6   roi1_left_rostral     1196 non-null   float64
 7   roi2_left_middle      1196 non-null   float64
 8   roi3_left_caudal      1196 non-null   float64
 9   roi4_right_rostral    1196 non-null   float64
 10  roi5_right_middle     1196 non-null   float64
 11  roi6_right_caudal     1196 non-null   float64
 12  ref1_left_reference   1196 non-null   float64
 13  ref2_right_reference  1196 non-null   float64
dtypes: float64(9), object(5)
memory usage: 130.9+ KB


None

----------------------------------
Number of times every participant attended the survey


count
4     229
8      30
12      2
16      1
Name: count, dtype: int64

----------------------------------
Detailed time of the survey:


Series([], Name: count, dtype: int64)

----------------------------------
Any repeated test in a single month or during screening? (#2, #3, etc. indicates repeated visits.)


visit_name
LOG    1196
Name: count, dtype: int64

As the DTI table contains different measurements, pivotting it will be helpful. We observe that there might be several tests on the same variable, and we only keep the first one, as that is the most universal to all participants.

In [ ]:
def appropriate_pivotting(df):
    df=df.drop(["GUID", "visit_name", "visit_month","dti_brain_tissue"], axis=1)
    df_pivotted=df.pivot(columns=["dti_measure"], index="participant_id")
    df_pivotted.columns = ['_'.join(map(str, col)).strip() for col in df_pivotted.columns.values]
    cols=df_pivotted.columns
    for col in cols:
        if "#" in col:
            df_pivotted=df_pivotted.drop(col, axis=1)
    return df_pivotted

DTI_data_cleaned_and_pivotted=appropriate_pivotting(DTI_data_cleaned)

In [ ]:
DTI_data_cleaned_and_pivotted

,roi1_left_rostral_Eigenvalue1,roi1_left_rostral_Eigenvalue2,roi1_left_rostral_Eigenvalue3,roi1_left_rostral_Fractional Anisotropy,roi2_left_middle_Eigenvalue1,roi2_left_middle_Eigenvalue2,roi2_left_middle_Eigenvalue3,roi2_left_middle_Fractional Anisotropy,roi3_left_caudal_Eigenvalue1,roi3_left_caudal_Eigenvalue2,...,roi6_right_caudal_Eigenvalue3,roi6_right_caudal_Fractional Anisotropy,ref1_left_reference_Eigenvalue1,ref1_left_reference_Eigenvalue2,ref1_left_reference_Eigenvalue3,ref1_left_reference_Fractional Anisotropy,ref2_right_reference_Eigenvalue1,ref2_right_reference_Eigenvalue2,ref2_right_reference_Eigenvalue3,ref2_right_reference_Fractional Anisotropy
participant_id,,,,,,,,,,,,,,,,,,,,,
PP-3101,0.000926,0.000604,0.000387,0.327429,0.000877,0.000511,0.000337,0.364471,0.000812,0.000587,...,0.000463,0.317038,0.001470,0.000537,0.000258,0.565928,0.001434,0.000411,0.000243,0.603173
PP-3102,0.000926,0.000644,0.000376,0.327140,0.000933,0.000591,0.000334,0.368317,0.000967,0.000637,...,0.000391,0.365978,0.001348,0.000393,0.000153,0.632944,0.001318,0.000269,0.000116,0.685334
PP-3104,0.000969,0.000667,0.000444,0.296356,0.000942,0.000644,0.000365,0.340615,0.001015,0.000665,...,0.000533,0.253402,0.001319,0.000355,0.000221,0.612216,0.001410,0.000370,0.000213,0.624466
PP-3105,0.000743,0.000422,0.000188,0.450384,0.000654,0.000409,0.000164,0.439363,0.000950,0.000638,...,0.000588,0.235125,0.001318,0.000230,0.000039,0.728970,0.001533,0.000422,0.000207,0.627681
PP-3106,0.001083,0.000752,0.000571,0.255552,0.000935,0.000555,0.000411,0.329358,0.000886,0.000510,...,0.000307,0.377322,0.001405,0.000504,0.000220,0.579911,0.001460,0.000452,0.000197,0.612912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PP-4084,0.000964,0.000599,0.000482,0.288346,0.000963,0.000565,0.000449,0.316809,0.000973,0.000689,...,0.000381,0.334611,0.001522,0.000460,0.000331,0.569118,0.001427,0.000329,0.000142,0.667300
PP-4085,0.000837,0.000611,0.000314,0.342581,0.000952,0.000640,0.000400,0.322198,0.001055,0.000689,...,0.000318,0.370197,0.001434,0.000280,0.000117,0.692710,0.001349,0.000414,0.000195,0.608474
PP-4086,0.001096,0.000778,0.000549,0.267581,0.000977,0.000731,0.000358,0.346608,0.001114,0.000847,...,0.000413,0.325613,0.001451,0.000437,0.000194,0.617197,0.001703,0.000468,0.000259,0.618275


In [ ]:
DTI_data_cleaned_and_pivotted=DTI_data_cleaned_and_pivotted.reset_index()

In [ ]:
DTI_data_cleaned_and_pivotted.participant_id.value_counts().value_counts()

count
1    262
Name: count, dtype: int64

In [ ]:
cleaned_data = cleaned_data.merge(
    DTI_data_cleaned_and_pivotted,
    on="participant_id", 
    how="left"
)    

    

In [ ]:
display(cleaned_data[cleaned_data.participant_id=="BF-1045"])

,participant_id,visit_name,visit_month,mds_updrs_part_i_summary_score,mds_updrs_part_ii_summary_score,mds_updrs_part_iii_summary_score,caff_drinks_ever_used_regularly,caff_drinks_current_use,age_at_baseline,sex,...,roi6_right_caudal_Eigenvalue3,roi6_right_caudal_Fractional Anisotropy,ref1_left_reference_Eigenvalue1,ref1_left_reference_Eigenvalue2,ref1_left_reference_Eigenvalue3,ref1_left_reference_Fractional Anisotropy,ref2_right_reference_Eigenvalue1,ref2_right_reference_Eigenvalue2,ref2_right_reference_Eigenvalue3,ref2_right_reference_Fractional Anisotropy
1012,BF-1045,M0,0,7.0,12.0,39.0,NaN,NaN,72,Male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
cleaned_data.to_csv("cleaned_data.csv", index=False)